Now to account for the interaction term, we expand to include 2nd Degree polynomial features. The below equation represents the formulation.

\begin{align}
\hat{y}(\textbf{x}) = w_{0} + \sum_{i=1}^{n} w_{i} x_{i} +  \sum_{i=1}^n \sum_{j=i+1}^n w_{ij} x_{i} x_{j}
\end{align}

The challenge with solving the above equation is that the time complexity is $O(n^2)$

In order to work with this, we use a matrix factorization technique for the interaction terms, inspired by Matrix factorization. We introduce a hyperpameter K which represent the latent factors for factorizing the weight vector $w_{ij}$.

\begin{align}
\hat{y}(\textbf{x}) = w_{0} + \sum_{i=1}^{n} w_{i} x_{i} + \sum_{i=1}^{n} \sum_{j=i+1}^n \langle \textbf{v}_i , \textbf{v}_{j} \rangle x_i x_{j}
\end{align}


Using the computation specified in Stephen Rendles paper, we can simplify the interaction term to the below equation.

\begin{align}
\sum_{i=1}^n \sum_{j=i+1}^n \langle \textbf{v}_i, \textbf{v}_{j} \rangle x_{i} x_{j}
&= \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n \langle \textbf{v}_i, \textbf{v}_{j} \rangle x_{i} x_{j} - \frac{1}{2} \sum_{i=1}^n \langle \textbf{v}_i , \textbf{v}_{i} \rangle x_{i} x_{i}  \\
&= \frac{1}{2}\left(\sum_{i=1}^n \sum_{j=1}^n \sum_{f=1}^k v_{i,f} v_{j,f} x_{i} x_{j} \right)\frac{1}{2}\left( \sum_{i=1}^n \sum_{f=1}^k v_{i,f} v_{i,f} x_{i} x_{i} \right) \\
&= \frac{1}{2}\left(\sum_{i=1}^n \sum_{j=1}^n \sum_{f=1}^k v_{i,f} v_{j,f} x_{i} x_{j}  -  \sum_{i=1}^n \sum_{f=1}^k v_{i,f} v_{i,f} x_{i} x_{i} \right) \\
&= \frac{1}{2} \sum_{f=1}^{k} \left( \left(\sum_{i=1}^n v_{i,f}x_{i} \right) \left( \sum_{j=1}^n v_{j,f}x_{j} \right) - \sum_{i=1}^{n} v_{i,f}^2 x_{i}^2 \right) \\
&= \frac{1}{2} \sum_{f=1}^{k} \left( \left( \sum_{i}^{n} v_{i,f}x_{i} \right)^2  - \sum_{i=1}^{n} v_{i,f}^2 x_{i}^2 \right)
\end{align}

So, we can rewrite the equation to compute in $O(n)$ as
\begin{align}
\hat{y}(\textbf{x}) = w_{0} + \sum_{i=1}^{n} w_{i} x_{i} + \frac{1}{2} \sum_{f=1}^{k} \left( \left( \sum_{i}^{n} v_{i,f}x_{i} \right)^2  - \sum_{i=1}^{n} v_{i,f}^2 x_{i}^2 \right)
\end{align}


For our classification problem, we can define the gradients as :

\begin{align}
\frac{\partial}{\partial\theta}\hat{y}(\textbf{x}) =
\begin{cases}
1,  & \text{if $\theta$ is $w_0$} \\
x_i, & \text{if $\theta$ is $w_i$} \\
x_i\sum_{j=1}^{n} v_{j,f}x_j - v_{i,f}x_{i}^2 & \text{if $\theta$ is $v_{i,f}$}
\end{cases}
\end{align}

\begin{align}
\frac{\partial}{\partial x}\hat{y}(\textbf{x}) =
\frac{d}{dx}\left[ \ln \big(e^{-yx} + 1 \big) \right] 
&= \frac{1}{e^{-yx} + 1} \cdot  \frac{d}{dx}\left[e^{-yx} + 1 \right] \\
&= -\frac{y}{e^{yx} + 1}
\end{align}


In [1]:
from pyspark.mllib.regression import LabeledPoint
from pyspark import SparkContext, SparkConf
from pyspark.mllib.util import MLUtils
from pyspark.storagelevel import *
import pyspark.mllib.linalg
import numpy as np
from sklearn.metrics import auc, roc_curve, average_precision_score, log_loss, mean_squared_error
import time
import pickle
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [2]:
%matplotlib inline
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize']=(16.0, 12.0)

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# create Spark Session
from pyspark.sql import SparkSession
app_name = "final_project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [ ]:
x_train = [[(1,1,2,1,0),1],[(1,0,2,1,0),0],[(0,1,2,1,0),0],[(1,3,2,1,0),1],[(1,1,2,1,0),0],[(1,1,5,1,0),1],[(1,4,2,1,1),0]]
trainRDD = sc.parallelize(x_train)

In [128]:
def fmLoss(dataRDD, w) :
    """
    Computes the logloss given the data and model W
    dataRDD - array of features, label
    """
    w_bc = sc.broadcast(w)
    def probability_value(x,W): 
        xa = np.array([x])
        V =  xa.dot(W)
        V_square = (xa*xa).dot(W*W)
        phi = 0.5*(V*V - V_square).sum()
        return 1.0/(1.0 + np.exp(-phi))
    
    loss = dataRDD.map(lambda x:  (probability_value(x[0],w_bc.value), x[1])).map(lambda x: -(x[1] * np.log(x[0]) - (1-x[1])*np.log(1-x[0]))).mean()
    
    return loss

In [142]:
def fmGradUpdate(dataRDD, w, alpha, regParam):
    """
    Computes the gradient and updates the model
    """
    
    G = np.zeros(np.shape(w))
    w_bc = sc.broadcast(w)
    a_bc = sc.broadcast(alpha)
    def row_grad(x, y, W, regParam):
        xa = np.array([x])
        x_matrix = xa.T.dot(xa)
        VX =  xa.dot(W)
        VX_square = (xa*xa).dot(W*W)
        phi = 0.5*(VX*VX - VX_square).sum()
        expnyt = np.exp(y*phi)
        result = (-y)/(1+expnyt)* (np.dot(xa, W))
        
        return regParam*W + result
      
    grad = dataRDD.map(lambda x: (1, row_grad(x[0], x[1], w_bc.value, regParam))).reduceByKey(lambda x,y: np.add(x,y))
    Grad = grad.map(lambda x: (1,np.square(x[1]))).reduceByKey(lambda x,y: np.add(x,y))
    model = w - (alpha * (grad.values().collect()[0]) )/ np.sqrt(Grad.values().collect()[0])
    
    return model
    

In [143]:
model = fmGradUpdate(trainRDD, wInit, 0.01, 0.01)
print(wInit)
print(model)

[[-1.02685949 -0.66814272]
 [-0.99975465 -0.66884647]
 [-1.0101119  -0.66776546]
 [-1.01337432 -0.63827766]
 [-1.00511465 -0.64297301]]
[[0.04492211 0.1114009 ]
 [0.43213412 0.10134743]
 [0.28417333 0.11679029]
 [0.23756734 0.5380447 ]
 [0.35556262 0.47096824]]
[[0.05492211 0.1214009 ]
 [0.44213412 0.11134743]
 [0.29417333 0.12679029]
 [0.24756734 0.5480447 ]
 [0.36556262 0.48096824]]


In [121]:
def GradientDescent(trainRDD, testRDD, wInit, nSteps = 20, 
                    learningRate = 0.01, regParam = 0.01, verbose = False):
    """
    Perform nSteps iterations of OLS gradient descent and 
    track loss on a test and train set. Return lists of
    test/train loss and the models themselves.
    """
    # initialize lists to track model performance
    train_history, test_history, model_history = [], [], []
    
    # perform n updates & compute test and train loss after each
    model = wInit
    for idx in range(nSteps): 
        
        ############## YOUR CODE HERE #############
        model = fmGradUpdate(trainRDD, model, learningRate, regParam)
        training_loss = fmLoss(trainRDD, model) 
        test_loss = fmLoss(testRDD, model) 
        ############## (END) YOUR CODE #############
        
        # keep track of test/train loss for plotting
        train_history.append(training_loss)
        test_history.append(test_loss)
        model_history.append(model)
        
        # console output if desired
        if verbose:
            print("----------")
            print(f"STEP: {idx+1}")
            print(f"training loss: {training_loss}")
            print(f"test loss: {test_loss}")
            print(f"Model: {[k for k in model]}")
    return train_history, test_history, model_history

In [17]:
def wInitialization(dataRDD, factor):
    nrFeat = len(trainRDD.first()[0])
    np.random.seed(int(time.time())) 
    w =  np.random.ranf((nrFeat, factor))
    w = w / np.sqrt((w*w).sum())
    
    return w

In [146]:
start = time.time()
wInit = wInitialization(trainRDD, 2)
logerr_train, logerr_test, models = GradientDescent(trainRDD, trainRDD, wInit, nSteps = 50,
                                                    learningRate = 0.008, regParam = 0.01, verbose = True)
print(f"\n... trained {len(models)} iterations in {time.time() - start} seconds")

[[-0.39320266 -0.60822455]
 [-0.40194714 -0.62841343]
 [-0.398838   -0.60968196]
 [-0.41437717 -0.60547251]
 [-0.41587132 -0.60513439]]
----------
STEP: 1
training loss: -1.1433005523978539
test loss: -1.1433005523978539
Model: [array([0.34636381, 0.42599445]), array([0.22144265, 0.13758193]), array([0.265859  , 0.40517433]), array([0.04387084, 0.46530939]), array([0.02252584, 0.47013962])]
[[-0.36361933 -0.55725663]
 [-0.37236381 -0.5774455 ]
 [-0.36925466 -0.55871403]
 [-0.38479383 -0.55450458]
 [-0.38628798 -0.55416646]]
----------
STEP: 2
training loss: -1.2029827884266122
test loss: -1.2029827884266122
Model: [array([0.35436381, 0.43399445]), array([0.22944265, 0.14558193]), array([0.273859  , 0.41317433]), array([0.05187084, 0.47330939]), array([0.03052584, 0.47813962])]
[[-0.33468252 -0.50859341]
 [-0.343427   -0.52878228]
 [-0.34031786 -0.51005081]
 [-0.35585703 -0.50584136]
 [-0.35735118 -0.50550324]]
----------
STEP: 3
training loss: -1.26428226594989
test loss: -1.2642822659

In [19]:
print(wInit)

[[0.06795161 0.03869574]
 [0.0912008  0.38786739]
 [0.51665977 0.33872864]
 [0.47325027 0.01188713]
 [0.24568004 0.41107944]]
